In [56]:
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

import imageio as img

import math

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu

In [86]:
# setting variables for the directory to the data
todata = "/home/abdullah/Desktop/Abdullah/LUMS/Senior/Sproj/ImgPreProcessing/Channel/"
moredata = "/home/abdullah/Desktop/Abdullah/LUMS/Senior/Sproj/ImgPreProcessing/AmplifiedDataset"


training_iters = 10 
learning_rate = 0.0001 
batch_size = 128

# data input (img shape: 28*28)
n_input = 28

# total classes (0-9 digits)
n_classes = 26

In [3]:
# saving current directory
cur = os.getcwd()

# changing directoy to data set
os.chdir(todata)
# getting all the folder names
nmes = os.listdir(".")
nmes.sort()

# reading all the data into labels and data numpy arrays
labels = []
data = []
for letr in nmes:
    for file in os.listdir(todata+letr):
        labels.append(letr)
        f = img.imread(todata + letr + "/"+ file)
        data.append(f)

        # change back directory
os.chdir(cur)

print("\nData labales matrix")
labels = np.array(labels)
print(labels, labels.dtype, labels.shape)

# diving by 255 to get clamp values between zero and one
print("\nData matrix")
data = np.array(data, np.float32)/255
print(data.dtype, data.shape)



Data labales matrix
['A' 'A' 'A' ... 'Z' 'Z' 'Z'] <U1 (3844,)

Data matrix
float32 (3844, 28, 28)


In [4]:
# converting labels into onehot encodings

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
# print(integer_encoded)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)

(3844, 26)


/home/abdullah/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
# function that splits up the data

def train_val_split(data, labels, keep):
  

    data = np.array(data)
    labels = np.array(labels)

    #split the data
    number_of_validation_points = data.shape[0]

    split = int(number_of_validation_points*keep)

    # getting curent state so shuffle is the same for both arrays
    curState = np.random.get_state()
    np.random.shuffle(data)
    # setting the state
    np.random.set_state(curState)
    np.random.shuffle(labels)



    #images are unflattened
    temp_val = data[split:]
    val_label = labels[split:]
    temp_train = data[:split]
    train_label = labels[:split]

    val = temp_val
    train = temp_train
    
    train = np.array(train)
    val = np.array(val)

    print("The splits are: ")
    print(train.shape)   
    print(train_label.shape)
    print(val.shape) 
    print(val_label.shape)

    return train.T, train_label, val.T, val_label



In [6]:
# splitting up the data
tr, tr_l, vl, vl_l = train_val_split(data, onehot_encoded, 0.8)



The splits are: 
(3075, 28, 28)
(3075, 26)
(769, 28, 28)
(769, 26)


In [27]:
# Reshape training and testing image
train_X = tr.reshape(-1, n_input, n_input, 1)
test_X = vl.reshape(-1,n_input,n_input, 1)



train_y = tr_l
test_y = vl_l

train_X.shape, test_X.shape, train_y.shape, test_y.shape

((3075, 28, 28, 1), (769, 28, 28, 1), (3075, 26), (769, 26))

In [115]:
# tensorflow variables will be assigned inside the functions

# simple convolution layer
def conv_layer(inp, ch_in, ch_out, name="conv", strides=1, k=2):
    # giving the same naming structure for graph in tensorboard
    with tf.name_scope(name):
        # the first two are kernel dimen then its the input depth and 
        # then the number of filters you want ie output depth
        w = tf.Variable(tf.truncated_normal([5, 5, ch_in, ch_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[ch_out]), name="B")

        # applying the kernel
        conv = tf.nn.conv2d(inp, w, strides=[1, strides, strides,1], padding="SAME")

        # activation
        act = tf.nn.relu(conv + b)

        return tf.nn.max_pool(act, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


# fully connected layer
def fc_layer(inp, ch_in, ch_out, name="fulluConnected"):
    # giving the same naming structure for graph in tensorboard
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([ch_in, ch_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[ch_out]), name="B")

        # linear operation and activation
        act = tf.nn.relu(tf.matmul(inp, w) + b)

        return act


def maxpool_layer(x, name="maxpool", k=2):
    # giving the same naming structure for graph in tensorboard
    with tf.name_scope(name):
        return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [116]:
# the whole compuational graph/network
def conv_net(x):  

    # here we call the convolution layer we had defined above and pass the input image x
    # since image is grascale input channel = 1 and output we specified to 32 kernels
    conv1 = conv_layer(x, 1, 32, "conv1")
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 14*14 matrix.
#     pool1 = maxpool_layer(conv1, "maxpool1", 2)


    # here we call the convolution layer we had defined above.
    conv2 = conv_layer(conv1, 32, 64, "conv2")
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matrix.
#     pool2 = maxpool_layer(conv2, "maxpool2", 2)

    conv3 = conv_layer(conv2, 64, 128, "conv3")
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
#     pool3 = maxpool_layer(conv3, "maxpool3", 2)


    # flatten the ouput of pool for fully connected
    # Reshape output to fit fully connected layer input
    flat = tf.reshape(conv3, [-1, math.ceil(n_classes/8)*math.ceil(n_classes/8)*128])

    # Fully connected layer
    fc1 = fc_layer(flat, math.ceil(n_classes/8)*math.ceil(n_classes/8)*128, 1024, "fc1")

    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = fc_layer(fc1, 1024, n_classes,"fc2")

    return out

In [121]:
tf.reset_default_graph()
sess = tf.Session()# telling the session to initialize the variables


# input dimen and output dimen
x = tf.placeholder("float", [None, n_input,n_input, 1], name="x")
y = tf.placeholder("float", [None, n_classes], name = "labels")

# storing the graph in the variable pred
pred = conv_net(x)

# cost fucntion for the graph
with tf.name_scope("entropy"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

# optiizr for the network
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

with tf.name_scope("accuracy"):
    #Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

    #calculate accuracy across all the given images and average them out. 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
sess.run(init) 

# storing the values in lists
train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

summary_writer = tf.summary.FileWriter('./Output')
summary_writer.add_graph(sess.graph)
for i in range(training_iters):
    for batch in range(len(train_X)//batch_size):
        batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
        batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))]    
        # Run optimization op (backprop).
        # Calculate batch loss and accuracy
        opt = sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y})

    print("Iter " + str(i) + ", Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
    print("Optimization Finished!")

    # Calculate accuracy for all test images
    test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: test_X,y : test_y})
    train_loss.append(loss)
    test_loss.append(valid_loss)
    train_accuracy.append(acc)
    test_accuracy.append(test_acc)
    print("Testing Accuracy:","{:.5f}".format(test_acc))
summary_writer.close()

Iter 0, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
Iter 1, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
Iter 2, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
Iter 3, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
Iter 4, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
Iter 5, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
Iter 6, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
Iter 7, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
Iter 8, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
Iter 9, Loss= 3.258096, Training Accuracy= 0.04688
Optimization Finished!
Testing Accuracy: 0.04421
